In [18]:
#import libraries
import os
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [22]:
#get password from environment variables
password = os.getenv('password')

In [23]:
# Variables
host = 'localhost'
database = 'postgres' #default
user = 'postgres'
port = '5432' #default

In [24]:
#Connect to postgres database 
conn = psycopg2.connect(host=host,
                         database=database,
                         user=user,
                         password=password,
                         port=port)

# Set isolation level to autocommit
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)

cursor = conn.cursor()

In [25]:
# Create a new database
cursor.execute("CREATE DATABASE population")

conn.commit()
cursor.close()
conn.close()

In [34]:
#New database name
database = 'population'

In [27]:
# Create a new engine to connect to database
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

In [28]:
#Declare a new table_name of choice
table_name = 'population_data'

In [32]:
# Let's read the data with pandas
df = pd.read_csv('world_population.csv')
df.head(2) #show first 2 rows only

,Rank,CCA3,Country/Territory,Capital,Continent,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage
0,36,AFG,Afghanistan,Kabul,Asia,41128771,38972230,33753499,28189672,19542982,10694796,12486631,10752971,652230,63.0587,1.0257,0.52
1,138,ALB,Albania,Tirana,Europe,2842321,2866849,2882481,2913399,3182021,3295066,2941651,2324731,28748,98.8702,0.9957,0.04


In [33]:
# Send new data to the new postgres table(population_data)

df.to_sql(table_name,engine, index=False, if_exists='replace')

234

In [35]:
# Connect to the new database to query the new table
conn = psycopg2.connect(host=host,
                         database=database,
                         user=user,
                         password=password,
                         port=port)

# Set isolation level to autocommit
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)

cursor = conn.cursor()

In [49]:
cursor.execute('SELECT "2000 Population" FROM population_data\
               LIMIT 5')

data = cursor.fetchall()
for row in data:
    print(row)

(19542982,)
(3182021,)
(30774621,)
(58230,)
(66097,)


In [50]:
# Close all sessions carefully.

engine.dispose()
cursor.close()
conn.close()